# Credit Card Customer Segmentation & Profit Optimization Strategy

**Notebook: ETL (Extract, Transform, Load)**

In this notebook, we:
- **Extract**: Load the raw dataset from CSV
- **Transform**: Clean missing values and standardize data types  
- **Load**: Save the cleaned dataset for analysis

This step focuses on basic data preparation - ensuring the dataset is clean and ready for exploratory data analysis.

In [26]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("../data/credit_card_data.csv")
# Preview shape and sample
print(f"Rows: {df.shape[0]:,} | Columns: {df.shape[1]}")
df.head()

Rows: 30,000 | Columns: 26


,Customer_ID,Age,Gender,Marital_Status,Education_Level,Annual_Income,Income_Bracket,Credit_Score,Default_Risk_Score,Tenure_Months,...,Late_Payments,Credit_Utilization,APR,Annual_Fee,Rewards_Earned,Rewards_Redeemed,Profit_Contribution,State,Employment_Status,Dependents
0,CUST38862,32,Male,Married,High School,57905.39,Medium,759,0.022,157,...,1.0,0.13,16.54,0,23.84,17.55,-28.93,TX,Full-time,0
1,CUST19624,37,Male,Divorced,Associate,14433.53,Low,478,0.304,99,...,3.0,0.14,28.31,199,1.88,1.35,8.06,CA,Part-time,3
2,CUST60456,28,Male,Married,Associate,54316.13,Medium,721,0.075,34,...,0.0,0.15,18.90,0,40.65,37.48,-59.38,OH,Full-time,1
3,CUST45541,69,Female,Single,Master,70442.34,Medium,769,0.023,140,...,0.0,0.11,13.32,0,28.20,18.64,-33.68,MI,Retired,0
4,CUST60708,38,Female,Married,Associate,77006.13,Medium,696,0.055,195,...,3.0,0.12,21.18,125,9.17,7.36,-3.62,Other,Full-time,3


## Step 1: Inspect Missing Values and Dtypes

We review nulls and data types for standardization, fixing only where necessary — no new features are created at this stage.

In [11]:
# Check column data types and nulls
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Customer_ID          30000 non-null  object 
 1   Age                  30000 non-null  int64  
 2   Gender               30000 non-null  object 
 3   Marital_Status       30000 non-null  object 
 4   Education_Level      29700 non-null  object 
 5   Annual_Income        30000 non-null  float64
 6   Income_Bracket       30000 non-null  object 
 7   Credit_Score         30000 non-null  int64  
 8   Default_Risk_Score   30000 non-null  float64
 9   Tenure_Months        30000 non-null  int64  
 10  Card_Type            30000 non-null  object 
 11  Credit_Limit         30000 non-null  float64
 12  Avg_Monthly_Spend    30000 non-null  float64
 13  Payer_Type           30000 non-null  object 
 14  Interest_Paid        30000 non-null  float64
 15  Payment_Behavior     30000 non-null 

In [29]:
# Check for missing values
missing = df.isnull().sum()
print("Missing Columns:")
print(missing[missing > 0])

Missing Columns:
Series([], dtype: int64)


In [28]:
# filling missing values
df['Education_Level'] = df['Education_Level'].fillna('Unknown')
df['Late_Payments'] = df['Late_Payments'].fillna(0)

In [30]:
# Display summary statistics for all numeric columns
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,30000.0,51.649433,19.709866,18.000,34.0000,52.000,69.0000,85.000
Annual_Income,30000.0,69964.043195,45688.606423,7508.380,35132.3925,62562.430,94381.1550,365942.590
Credit_Score,30000.0,697.963533,100.332764,344.000,634.0000,717.000,772.0000,850.000
Default_Risk_Score,30000.0,0.099998,0.095933,0.009,0.0280,0.065,0.1420,0.382
Tenure_Months,30000.0,102.777400,70.114644,1.000,40.0000,94.000,161.0000,240.000
Credit_Limit,30000.0,11769.997479,12455.266806,300.200,2617.1400,7423.930,14106.8675,49971.740
Avg_Monthly_Spend,30000.0,4794.255271,6546.463148,17.110,760.1775,2157.925,5958.0325,44133.990
Interest_Paid,30000.0,60.083755,90.664261,0.000,0.0000,22.845,82.2325,759.570
Payment_Behavior,30000.0,0.862686,0.128377,0.500,0.8100,0.900,0.9700,1.000
Late_Payments,30000.0,2.564300,1.918034,0.000,1.0000,2.000,4.0000,12.000


In [31]:
# Save cleaned dataset
df.to_csv('../data/cleaned_credit_data.csv', index=False)
print("Cleaned data saved to ../data/cleaned_credit_data.csv")

Cleaned data saved to ../data/cleaned_credit_data.csv


In [32]:
# Final ETL Verification
print("ETL PROCESS COMPLETE")
print("=" * 25)
print(f"Final Dataset: {df.shape[0]:,} rows, {df.shape[1]} columns")
print(f" Missing Values: {df.isnull().sum().sum()}")
print(f"Saved to: ../data/cleaned_credit_data.csv")

ETL PROCESS COMPLETE
Final Dataset: 30,000 rows, 26 columns
 Missing Values: 0
Saved to: ../data/cleaned_credit_data.csv


## Step 2: Load Data into SQLite Database

Export the cleaned dataset to SQLite for SQL-based analysis and exploration.

In [ ]:
import sqlite3

# Create SQLite database and load the cleaned dataframe
db_path = '../data/credit_card_database.db'
conn = sqlite3.connect(db_path)

# Load cleaned dataframe into SQLite table
df.to_sql('customers', conn, if_exists='replace', index=False)
print(f"Data successfully loaded into SQLite database at {db_path}")


Data successfully loaded into SQLite database at ../data/credit_card_database.db


In [34]:
# Preview the first 5 rows from the SQL table
pd.read_sql_query("SELECT * FROM customers LIMIT 5", conn)

,Customer_ID,Age,Gender,Marital_Status,Education_Level,Annual_Income,Income_Bracket,Credit_Score,Default_Risk_Score,Tenure_Months,...,Late_Payments,Credit_Utilization,APR,Annual_Fee,Rewards_Earned,Rewards_Redeemed,Profit_Contribution,State,Employment_Status,Dependents
0,CUST38862,32,Male,Married,High School,57905.39,Medium,759,0.022,157,...,1.0,0.13,16.54,0,23.84,17.55,-28.93,TX,Full-time,0
1,CUST19624,37,Male,Divorced,Associate,14433.53,Low,478,0.304,99,...,3.0,0.14,28.31,199,1.88,1.35,8.06,CA,Part-time,3
2,CUST60456,28,Male,Married,Associate,54316.13,Medium,721,0.075,34,...,0.0,0.15,18.90,0,40.65,37.48,-59.38,OH,Full-time,1
3,CUST45541,69,Female,Single,Master,70442.34,Medium,769,0.023,140,...,0.0,0.11,13.32,0,28.20,18.64,-33.68,MI,Retired,0
4,CUST60708,38,Female,Married,Associate,77006.13,Medium,696,0.055,195,...,3.0,0.12,21.18,125,9.17,7.36,-3.62,Other,Full-time,3


In [60]:
pd.read_sql_query("PRAGMA table_info(customers);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,Customer_ID,TEXT,0,None,0
1,1,Age,INTEGER,0,None,0
2,2,Gender,TEXT,0,None,0
3,3,Marital_Status,TEXT,0,None,0
4,4,Education_Level,TEXT,0,None,0
5,5,Annual_Income,REAL,0,None,0
6,6,Income_Bracket,TEXT,0,None,0
7,7,Credit_Score,INTEGER,0,None,0
8,8,Default_Risk_Score,REAL,0,None,0
9,9,Tenure_Months,INTEGER,0,None,0


## Step 3: SQL Analysis - Understanding the Business

Now that our data is loaded, let's start with basic SQL queries to understand our business performance.

### Query 1: Overall Business Health
First, let's get a high-level view of our customer base and total profitability.

In [38]:
query = """
SELECT 
    COUNT(*) AS total_customers,
    ROUND(SUM(Profit_Contribution), 2) AS total_profit,
    ROUND(AVG(Profit_Contribution), 2) AS avg_profit_per_customer
FROM customers;
"""
pd.read_sql_query(query, conn)

,total_customers,total_profit,avg_profit_per_customer
0,30000,-877194.12,-29.24


### Query 2: Profit by Customer Income Level
Let's see which income groups are performing better or worse.

In [37]:
# Query: Profit breakdown by income bracket
query = """
SELECT 
    Income_Bracket,
    COUNT(*) AS num_customers,
    ROUND(SUM(Profit_Contribution), 2) AS total_profit,
    ROUND(AVG(Profit_Contribution), 2) AS avg_profit
FROM customers
GROUP BY Income_Bracket
ORDER BY total_profit DESC;
"""
pd.read_sql_query(query, conn)

,Income_Bracket,num_customers,total_profit,avg_profit
0,Low,8722,-72252.67,-8.28
1,Medium,14774,-330326.26,-22.36
2,High,6504,-474615.19,-72.97


### Query 3: Payment Behavior Analysis
How do different payment patterns affect profitability?

In [ ]:
query = """
SELECT 
    CASE 
        WHEN Avg_Monthly_Payment > Credit_Limit * 0.8 THEN 'High Payer (>80% limit)'
        WHEN Avg_Monthly_Payment > Credit_Limit * 0.3 THEN 'Medium Payer (30-80% limit)'
        ELSE 'Low Payer (<30% limit)'
    END AS payment_behavior,
    COUNT(*) AS num_customers,
    ROUND(AVG(Profit_Contribution), 2) AS avg_profit
FROM customers
GROUP BY payment_behavior
ORDER BY avg_profit DESC;
"""
pd.read_sql_query(query, conn)

,Card_Type,num_customers,avg_profit
0,Signature,995,-71.61
1,Platinum,3236,-57.93
2,Gold,10213,-35.93
3,Standard,15556,-16.17


### Query 4: Card Type Performance
Which card products are the biggest problem?

In [ ]:
query = """
SELECT 
    Card_Type,
    COUNT(*) AS num_customers,
    ROUND(AVG(Profit_Contribution), 2) AS avg_profit
FROM customers
GROUP BY Card_Type
ORDER BY avg_profit DESC;
"""
pd.read_sql_query(query, conn)

,interest_group,num_customers,avg_profit
0,No Interest Paid,10773,-24.99
1,Pays Interest,19227,-31.62


### Query 5: Interest Payment Impact
Do customers who pay interest perform better?

In [70]:
query = """
SELECT 
    CASE 
        WHEN Interest_Paid = 0 THEN 'No Interest Paid'
        ELSE 'Pays Interest'
    END AS interest_group,
    COUNT(*) AS num_customers,
    ROUND(AVG(Profit_Contribution), 2) AS avg_profit
FROM customers
GROUP BY interest_group;
"""
pd.read_sql_query(query, conn)

,interest_group,num_customers,avg_profit
0,No Interest Paid,10773,-24.99
1,Pays Interest,19227,-31.62


### Query 6: What's Causing the Losses?
Since we're losing money, let's break down the revenue and cost components to find the root cause.

In [41]:
# Examine the components that likely make up profit calculation
query = """
SELECT 
    COUNT(*) as sample_size,
    ROUND(AVG(Interest_Paid), 2) as avg_interest_paid,
    ROUND(AVG(Annual_Fee), 2) as avg_annual_fee,
    ROUND(AVG(Rewards_Earned), 2) as avg_rewards_earned,
    ROUND(AVG(Rewards_Redeemed), 2) as avg_rewards_redeemed,
    ROUND(AVG(Late_Payments), 2) as avg_late_payments,
    ROUND(AVG(Profit_Contribution), 2) as avg_profit,
    ROUND(MIN(Profit_Contribution), 2) as min_profit,
    ROUND(MAX(Profit_Contribution), 2) as max_profit
FROM customers
LIMIT 1;
"""
pd.read_sql_query(query, conn)

,sample_size,avg_interest_paid,avg_annual_fee,avg_rewards_earned,avg_rewards_redeemed,avg_late_payments,avg_profit,min_profit,max_profit
0,30000,60.08,69.51,71.13,56.99,2.56,-29.24,-559.9,197.76


In [75]:
# Let's look at a few customer examples to understand the profit calculation
query = """
SELECT 
    Customer_ID,
    Card_Type,
    Interest_Paid,
    Annual_Fee,
    Rewards_Earned,
    Rewards_Redeemed,
    Profit_Contribution
FROM customers
ORDER BY Profit_Contribution DESC
LIMIT 5;
"""
print("A few customer examples (best performing):")
pd.read_sql_query(query, conn)

A few customer examples (best performing):


,Customer_ID,Card_Type,Interest_Paid,Annual_Fee,Rewards_Earned,Rewards_Redeemed,Profit_Contribution
0,CUST67938,Gold,737.15,0,446.07,399.03,197.76
1,CUST12972,Gold,603.83,0,338.57,333.54,172.01
2,CUST92672,Standard,621.47,0,363.41,278.29,169.78
3,CUST32606,Platinum,468.02,199,277.15,274.25,136.43
4,CUST25965,Platinum,399.65,199,224.65,145.03,129.25


In [71]:
# Compare with worst performing customers
query = """
SELECT 
    Customer_ID,
    Card_Type,
    Interest_Paid,
    Annual_Fee,
    Rewards_Earned,
    Rewards_Redeemed,
    Profit_Contribution
FROM customers
ORDER BY Profit_Contribution ASC
LIMIT 5;
"""
print("Compare with worst performing customers:")
pd.read_sql_query(query, conn)

Compare with worst performing customers:


,Customer_ID,Card_Type,Interest_Paid,Annual_Fee,Rewards_Earned,Rewards_Redeemed,Profit_Contribution
0,CUST03115,Gold,565.91,0,769.47,721.99,-559.90
1,CUST86154,Platinum,447.46,95,647.25,506.77,-462.53
2,CUST91519,Gold,561.46,95,707.05,624.70,-444.28
3,CUST72407,Gold,371.33,0,574.52,364.23,-437.55
4,CUST88443,Gold,411.84,0,700.41,597.16,-435.12


### Query 7: Customer Examples
Let's look at specific customer examples to understand the profit calculation better.

In [56]:
# Simple breakdown by card type
query = """
SELECT 
    Card_Type,
    COUNT(*) AS num_customers,
    ROUND(AVG(Profit_Contribution), 2) AS avg_profit
FROM customers
GROUP BY Card_Type
ORDER BY avg_profit DESC;
"""
pd.read_sql_query(query, conn)

,Card_Type,num_customers,avg_profit
0,Standard,15556,-16.17
1,Gold,10213,-35.93
2,Platinum,3236,-57.93
3,Signature,995,-71.61


## ETL Summary & Key Findings

### **Project Objective:**
*To develop a data-driven segmentation of credit card customers that identifies distinct groups based on demographics, credit risk, spending behavior, and product usage, enabling targeted strategies that optimize profitability and minimize credit risk.*

### **Critical Discovery - Business Crisis Identified:**
Through our SQL analysis, we've uncovered a **financial crisis**: 
- **Total Loss:** $877,194 across 30,000 customers (-$29.24 average per customer)
- **Root Cause:** Revenue streams insufficient to cover rewards costs
- **Revenue:** Interest ($60.08) + Fees ($69.51) = $129.59 average per customer
- **Costs:** Rewards Earned ($71.13) + Rewards Redeemed ($56.99) = $128.12 average per customer
- **Margin:** Only $1.47 before operational costs, defaults, and other expenses

### **Strategic Implications:**
This crisis transforms our segmentation approach from **optimization** to **survival**:
1. **Identify "least unprofitable" segments** for retention focus
2. **Find customer characteristics** that correlate with lower losses  
3. **Develop targeted strategies** to restructure relationships with different segments
4. **Create risk models** to prevent acquiring similar loss-making customers

### **Next Steps:**
- **EDA Phase:** Deep analysis of customer characteristics and profit drivers
- **Segmentation:** Cluster customers by loss severity and characteristics  
- **Strategy Development:** Create actionable turnaround recommendations

In [79]:
# Close database connection
conn.close()

print("ETL PHASE COMPLETE")
print("=" * 50)
print("Data cleaned and loaded to database")
print("Initial SQL analysis complete") 
print("Business crisis identified and documented")
print("Ready for EDA and segmentation analysis")
print("\nOutputs:")
print("   ../data/cleaned_credit_data.csv")
print("   ../data/credit_card_database.db")
print("\nKey Finding: $877K annual loss due to unsustainable rewards economics")
print("Next: EDA notebook for customer segmentation and turnaround strategy")

ETL PHASE COMPLETE
Data cleaned and loaded to database
Initial SQL analysis complete
Business crisis identified and documented
Ready for EDA and segmentation analysis

Outputs:
   ../data/cleaned_credit_data.csv
   ../data/credit_card_database.db

Key Finding: $877K annual loss due to unsustainable rewards economics
Next: EDA notebook for customer segmentation and turnaround strategy
